In [1]:
import pandas as pd
import numpy as np
from arch import arch_model

In [11]:
data = pd.read_csv("数据.csv")
data

,交易日期,日收盘价
0,2018-03-01,5340
1,2018-03-02,5266
2,2018-03-05,5254
3,2018-03-06,5256
4,2018-03-07,5212
...,...,...
1477,2024-03-29,8216
1478,2024-04-01,8328
1479,2024-04-02,8286
1480,2024-04-03,8570


In [12]:
data['交易日期'] = pd.to_datetime(data['交易日期'])
data.set_index('交易日期', inplace=True)

# 将数据分割为训练集和测试集
train_start = '2018-03-01'
train_end = '2022-02-28'
test_start = '2023-03-01'
test_end = '2024-04-08'

train_data = data[train_start:train_end]['日收盘价']
test_data = data[test_start:test_end]['日收盘价']

In [13]:
train_data

交易日期
2018-03-01     5340
2018-03-02     5266
2018-03-05     5254
2018-03-06     5256
2018-03-07     5212
              ...  
2022-02-22    10770
2022-02-23    11178
2022-02-24    11596
2022-02-25    11360
2022-02-28    11402
Name: 日收盘价, Length: 971, dtype: int64

In [14]:
test_data

交易日期
2023-03-01    8268
2023-03-02    8340
2023-03-03    8418
2023-03-06    8250
2023-03-07    8202
              ... 
2024-03-29    8216
2024-04-01    8328
2024-04-02    8286
2024-04-03    8570
2024-04-08    8218
Name: 日收盘价, Length: 268, dtype: int64

In [26]:
rolling_predictions = []

# 用滚动窗口预测每个月的波动率
for test_month_start in pd.date_range(test_start, test_end, freq='MS'):
    test_month_end = test_month_start + pd.offsets.MonthEnd(0)
    # 计算当前窗口内的训练数据的开始日期和结束日期
    train_end_window = test_month_start
    train_start_window = train_end_window - pd.offsets.MonthBegin(36)#向前移动一个月
    # 选择当前窗口内的训练数据
    train_set = data[train_start_window:train_end_window]['日收盘价']
    
    model = arch_model(train_set, vol='Garch', p=1, q=1)
    model_fit = model.fit()
    
    forecast = model_fit.forecast(horizon=1)
    forecast_variance = forecast.variance.values.squeeze() #forecast.variance.values 是预测的方差值，它是一个数组或矩阵
    #其中包含了模型在每个预测步长上的方差值。
    #squeeze() 方法用于从数组或矩阵中删除单维度的条目，这样就可以将其转换为一维数组。
    #在这个上下文中，forecast_variance 是一个一维数组，包含了每个预测步长上的方差值。
    rolling_predictions.append(forecast_variance)

# 输出每个月的波动率预测结果
print("Rolling Predictions:")
for prediction in rolling_predictions:
    print(prediction)

/Users/apple/anaconda3/lib/python3.11/site-packages/arch/univariate/base.py:311: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 3.585e+06. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 0.01 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(
/Users/apple/anaconda3/lib/python3.11/site-packages/arch/univariate/base.py:311: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 3.315e+06. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 0.01 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(
/Users/apple/anaconda3/lib/python3.11/site-packages/arch

Iteration:      1,   Func. Count:      6,   Neg. LLF: 22970.07175239892
Iteration:      2,   Func. Count:     12,   Neg. LLF: 6052.311490645563
Iteration:      3,   Func. Count:     17,   Neg. LLF: 6051.610751126228
Iteration:      4,   Func. Count:     22,   Neg. LLF: 6051.107539466952
Iteration:      5,   Func. Count:     27,   Neg. LLF: 6051.061910660601
Iteration:      6,   Func. Count:     32,   Neg. LLF: 6051.011776869026
Iteration:      7,   Func. Count:     37,   Neg. LLF: 6050.698347820091
Iteration:      8,   Func. Count:     42,   Neg. LLF: 6049.87533795492
Iteration:      9,   Func. Count:     47,   Neg. LLF: 6048.377195073791
Iteration:     10,   Func. Count:     52,   Neg. LLF: 6043.865460953558
Iteration:     11,   Func. Count:     57,   Neg. LLF: 6038.684284383511
Iteration:     12,   Func. Count:     62,   Neg. LLF: 6038.488810488951
Iteration:     13,   Func. Count:     67,   Neg. LLF: 6038.474686819094
Iteration:     14,   Func. Count:     72,   Neg. LLF: 6038.474608

Iteration:      9,   Func. Count:     47,   Neg. LLF: 5784.643041689473
Iteration:     10,   Func. Count:     52,   Neg. LLF: 5765.329905628856
Iteration:     11,   Func. Count:     57,   Neg. LLF: 5751.741161514134
Iteration:     12,   Func. Count:     62,   Neg. LLF: 5750.187616298403
Iteration:     13,   Func. Count:     67,   Neg. LLF: 5749.706607702143
Iteration:     14,   Func. Count:     72,   Neg. LLF: 5749.688417497554
Iteration:     15,   Func. Count:     77,   Neg. LLF: 5749.6881944263805
Iteration:     16,   Func. Count:     82,   Neg. LLF: 5749.688196922743
Optimization terminated successfully    (Exit mode 0)
            Current function value: 5749.688194163418
            Iterations: 16
            Function evaluations: 84
            Gradient evaluations: 16
Iteration:      1,   Func. Count:      6,   Neg. LLF: 22745.394184409088
Iteration:      2,   Func. Count:     12,   Neg. LLF: 5865.238082915695
Iteration:      3,   Func. Count:     17,   Neg. LLF: 5864.9404064419

/Users/apple/anaconda3/lib/python3.11/site-packages/arch/univariate/base.py:311: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 1.935e+06. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 0.01 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(
/Users/apple/anaconda3/lib/python3.11/site-packages/arch/univariate/base.py:311: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 1.876e+06. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 0.01 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(
/Users/apple/anaconda3/lib/python3.11/site-packages/arch

In [27]:
rolling_predictions

[array(163117.18275846),
 array(173362.32938297),
 array(1090084.14072955),
 array(2100363.01706379),
 array(297663.40564706),
 array(107648.23994576),
 array(59933.768463),
 array(136855.67766044),
 array(407646.93060652),
 array(330325.42265795),
 array(437659.18067265),
 array(373506.41060428),
 array(59614.86816605),
 array(353520.35344272)]